<a href="https://colab.research.google.com/github/Brainnext/Implicit-Neural-Representations-for-attention-cloud-point-data-for-3D-Reconstruction/blob/main/Implicit_Neural_Representations_for_attention_cloud_point_data_for_3D_Reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Installing and importing dependencies

In [1]:
pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.5/736.5 kB 17.4 MB/s eta 0:00:00


In [8]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import trimesh
import os
from tqdm import tqdm
from torch_geometric.nn import PointNetConv
from torch_geometric.nn.pool import fps
from torch_geometric.utils import to_dense_batch
from torch_geometric.data import Data # Needed for PyG operations

### Step 2: The Implicit Function Decoder (INR) Design

In [3]:
# Hyperparameters

L = 10 # Frequency count for positional encoding
Z_DIM = 512 # Dimension of the latent shape code (z
HIDDEN_DIM = 256 # Width of tHE MLP layers
NUM_LAYERS = 8 # Depth of the MLP

class PositionalEncoding(nn.Module):
  """
  Class for the postional encoding module
  """
  def __init__(self, L=10):
    super().__init__()
    # 3D points * 2(sin/cos) * L frequencies
    self.output_dim = 3 * 2 * L
    self.L = L

  def forward(self, X):
    embeds = []
    for i in range(self.L):
      freq_band = 2.**i *np.pi
      embeds.append(torch.sin(freq_band * x))
      embeds.append(torch.cos(freq_band * x))
    return torch.cat(embeds, dim=-1)

In [6]:
class SDFDecoder(nn.Module):
    def __init__(self, z_dim=Z_DIM, hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS, L=L):
        super().__init__()

        self.positional_encoder = PositionalEncoding(L=L)
        pos_dim = self.positional_encoder.output_dim

        # Initial layer takes Positional Encoded point + Latent Code
        self.in_dim = pos_dim + z_dim

        layers = []

        # 1. First layer
        layers.append(nn.Linear(self.in_dim, hidden_dim))
        layers.append(nn.Softplus(beta=100)) # High beta Softplus is preferred for SDF

        # 2. Main layers (Deep Network)
        for i in range(num_layers - 2):
            # Complex Note: We often concatenate the latent code 'z' again at intermediate layers
            # (a "skip connection" of the conditioning vector) to aid optimization.
            if i == (num_layers // 2) - 1: # Example: Concatenate z halfway
                 layers.append(nn.Linear(hidden_dim + z_dim, hidden_dim))
            else:
                 layers.append(nn.Linear(hidden_dim, hidden_dim))

            layers.append(nn.Softplus(beta=100))

        # 3. Output layer
        layers.append(nn.Linear(hidden_dim, 1)) # Output is the single SDF value

        self.net = nn.Sequential(*layers)
        self.z_dim = z_dim
        self.hidden_dim = hidden_dim

    def forward(self, p, z):
        # p: (B, N_query, 3), z: (B, Z_DIM)

        # 1. Positional Encoding of points
        p_enc = self.positional_encoder(p) # (B, N_query, pos_dim)

        # 2. Replicate and Concatenate Latent Code (z)
        # z must be expanded to match the number of query points (N_query)
        z_expanded = z.unsqueeze(1).repeat(1, p.shape[1], 1) # (B, N_query, Z_DIM)

        x = torch.cat([p_enc, z_expanded], dim=-1) # (B, N_query, pos_dim + Z_DIM)

        # Iterate through the network
        for i, layer in enumerate(self.net):
            if isinstance(layer, nn.Linear) and i > 0:
                # Re-concatenate latent code at the skip connection (e.g., halfway)
                if i == self.net[i].in_features - 1: # Crude way to find the skip layer
                    x = torch.cat([x, z_expanded], dim=-1)

            x = layer(x)

        # Final output is the predicted SDF (B, N_query, 1)
        return x.squeeze(-1)

In [7]:
def eikonal_loss(sdf_decoder, p_queries, z_codes):
    """
    Calculates the Eikonal Loss: ||∇F(p)||_2 - 1)^2
    This forces the gradient of the SDF to have unit magnitude (1).
    """

    # CRITICAL: We need gradients w.r.t. the input points 'p_queries'
    p_queries.requires_grad_(True)

    # 1. Forward pass to get SDF values
    # The output is (B * N_query)
    sdf_output = sdf_decoder(p_queries, z_codes)

    # Flatten the SDF output for gradient calculation
    sdf_output_flat = sdf_output.view(-1)

    # Create dummy tensor of ones for Jacobian (gradient) calculation
    ones = torch.ones_like(sdf_output_flat)

    # 2. Compute the gradient (∇F) using automatic differentiation
    # gradients will be shape (B * N_query, 3)
    gradients = torch.autograd.grad(
        outputs=sdf_output_flat,
        inputs=p_queries,
        grad_outputs=ones,
        create_graph=True,  # IMPORTANT: Needed to allow backprop through this loss term
        retain_graph=True,
        only_inputs=True
    )[0]

    # 3. Calculate the L2 norm (magnitude) of the gradient
    # The norm is calculated across the last dimension (x, y, z)
    gradient_norm = gradients.norm(2, dim=-1)

    # 4. Compute the Eikonal loss: (||∇F|| - 1)^2
    eikonal_loss_val = torch.mean((gradient_norm - 1)**2)

    return eikonal_loss_val